In [4]:
# If you have an NVIDIA GPU and cudf installed
#%load_ext cudf.pandas 

import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
import json
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf

%matplotlib inline

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

In [5]:
current_df = None

CIK_dict = pd.read_csv('../CIK_dict.csv', converters={'cik_str': str})
CIK_dict

,cik_str,ticker,title
0,0001045810,NVDA,NVIDIA CORP
1,0000789019,MSFT,MICROSOFT CORP
2,0000320193,AAPL,Apple Inc.
3,0001018724,AMZN,AMAZON COM INC
4,0001652044,GOOGL,Alphabet Inc.
...,...,...,...
10056,0002055896,FCHRF,Georg Fischer AG/ADR
10057,0001992829,SDZXF,SANDOZ GROUP AG
10058,0002053411,PCPPF,PC Partner Group Ltd/ADR
10059,0002051587,BSAAU,BEST SPAC I Acquisition Corp.


In [3]:
CIK = '0000320193'
headers = {
    'User-Agent': 'your_email@email.com'
}

url = f'https://data.sec.gov/submissions/CIK{CIK}.json'

response = requests.get(url, headers=headers)
data = response.json()

with open('data.json', 'w') as f:
    json.dump(data, f)

In [4]:
df = pd.DataFrame(data['filings']['recent'])

df[df['form'] == '10-K']

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
54,0000320193-24-000123,2024-11-01,2024-09-28,2024-11-01T10:01:36.000Z,34,10-K,001-36743,241416806,,XBRL,9759333,1,1,aapl-20240928.htm,10-K
148,0000320193-23-000106,2023-11-03,2023-09-30,2023-11-02T22:08:27.000Z,34,10-K,001-36743,231373899,,XBRL,9569569,1,1,aapl-20230930.htm,10-K
226,0000320193-22-000108,2022-10-28,2022-09-24,2022-10-27T22:01:14.000Z,34,10-K,001-36743,221338448,,XBRL,10332356,1,1,aapl-20220924.htm,10-K
301,0000320193-21-000105,2021-10-29,2021-09-25,2021-10-28T22:04:28.000Z,34,10-K,001-36743,211359752,,XBRL,10502096,1,1,aapl-20210925.htm,10-K
370,0000320193-20-000096,2020-10-30,2020-09-26,2020-10-29T22:06:25.000Z,34,10-K,001-36743,201273977,,XBRL,12502600,1,1,aapl-20200926.htm,10-K
441,0000320193-19-000119,2019-10-31,2019-09-28,2019-10-30T22:12:36.000Z,34,10-K,001-36743,191181423,,XBRL,12861616,1,1,a10-k20199282019.htm,10-K
511,0000320193-18-000145,2018-11-05,2018-09-29,2018-11-05T13:01:40.000Z,34,10-K,001-36743,181158788,,10-K,12275572,1,0,a10-k20189292018.htm,10-K
599,0000320193-17-000070,2017-11-03,2017-09-30,2017-11-03T12:01:37.000Z,34,10-K,001-36743,171174673,,10-K,14071062,1,0,a10-k20179302017.htm,10-K
744,0001628280-16-020309,2016-10-26,2016-09-24,2016-10-26T20:42:16.000Z,34,10-K,001-36743,161953070,,10-K,13277662,1,0,a201610-k9242016.htm,10-K
861,0001193125-15-356351,2015-10-28,2015-09-26,2015-10-28T20:31:09.000Z,34,10-K,001-36743,151180619,,10-K,9594425,1,0,d17062d10k.htm,FORM 10-K


In [ ]:
def get_financial_reports(Ticker):
    
    # Load CIK dictionary
    CIK = CIK_dict[CIK_dict['ticker'] == Ticker]['cik_str'].values[0]
    
    
    # Fetching submissions for target company from SEC
    headers = {
        'User-Agent': 'your_email@email.com'
    }

    url = f'https://data.sec.gov/submissions/CIK{CIK}.json'

    response = requests.get(url, headers=headers)
    data = response.json()

    df = pd.DataFrame(data['filings']['recent'])
    
    # Filtering for 10-K filings
    df_list = df[df['form'] == '10-K'].to_dict(orient='records')
    
    # Downloading 10-K filings
    for entry in df_list:
        accession_number = entry['accessionNumber'].replace('-', '')
        response = requests.get(f'https://www.sec.gov/Archives/edgar/data/{CIK}/{accession_number}/Financial_Report.xlsx', headers=headers)
        with open(f'{Ticker}_{entry['form']}_{entry['filingDate']}_Financial_Report.xlsx', 'wb') as f:
            f.write(response.content)

In [38]:
get_financial_reports('MSFT')

In [34]:
income_statement_names = ['Consolidated Statements of Inco',
                          'CONSOLIDATED STATEMENTS OF INCO',
                          'Consolidated Statements of Earn',
                          'Consolidated Statements of Oper',
                          'CONSOLIDATED STATEMENTS OF OPER',
                          'CONSOLIDATED_STATEMENTS_OF_OPE'
                          ]

for name in income_statement_names:
    try:
        df_excel = pd.read_excel('AAPL_10-K_2014-10-27_Financial_Report.xlsx', sheet_name=name)
        df_excel.to_csv(f'{name}.csv')
    except:
        print(f'{name} not found')
        pass

Consolidated Statements of Inco not found
CONSOLIDATED STATEMENTS OF INCO not found
Consolidated Statements of Earn not found
Consolidated Statements of Oper not found
CONSOLIDATED STATEMENTS OF OPER not found


In [40]:
df_excel = pd.read_excel('MSFT_10-K_2024-07-30_Financial_Report.xlsx', sheet_name=None)

df_excel.keys()

dict_keys(['Document and Entity Information', 'INCOME STATEMENTS', 'COMPREHENSIVE INCOME STATEMENTS', 'BALANCE SHEETS', 'BALANCE SHEETS (Parenthetical)', 'CASH FLOWS STATEMENTS', "STOCKHOLDERS' EQUITY STATEMENTS", 'Pay vs Performance Disclosure', 'Insider Trading Arrangements', 'Insider Trading Policies and Pr', 'ACCOUNTING POLICIES', 'EARNINGS PER SHARE', 'OTHER INCOME (EXPENSE), NET', 'INVESTMENTS', 'DERIVATIVES', 'INVENTORIES', 'PROPERTY AND EQUIPMENT', 'BUSINESS COMBINATIONS', 'GOODWILL', 'INTANGIBLE ASSETS', 'DEBT', 'INCOME TAXES', 'UNEARNED REVENUE', 'LEASES', 'CONTINGENCIES', "STOCKHOLDERS' EQUITY", 'ACCUMULATED OTHER COMPREHENSIVE', 'EMPLOYEE STOCK AND SAVINGS PLAN', 'SEGMENT INFORMATION AND GEOGRAP', 'ACCOUNTING POLICIES (Policies)', 'ACCOUNTING POLICIES (Tables)', 'EARNINGS PER SHARE (Tables)', 'OTHER INCOME (EXPENSE), NET (Ta', 'INVESTMENTS (Tables)', 'DERIVATIVES (Tables)', 'INVENTORIES (Tables)', 'PROPERTY AND EQUIPMENT (Tables)', 'BUSINESS COMBINATIONS (Tables)', 'GOODWIL

In [1]:
# Making standardized statements

statement_dict = {
    'AAPL': ['CONSOLIDATED_STATEMENTS_OF_OPE', 'CONSOLIDATED_BALANCE_SHEETS', 'CONSOLIDATED_STATEMENTS_OF_CAS'],
    'MSFT': ['INCOME STATEMENTS', 'BALANCE SHEETS', 'CASH FLOWS STATEMENTS' ],
    'NVDA': ['CONSOLIDATED STATEMENTS OF INCO', 'CONSOLIDATED BALANCE SHEETS', 'CONSOLIDATED STATEMENTS OF CASH'],
    'AMD': ['CONSOLIDATED STATEMENTS OF INCO', 'CONSOLIDATED BALANCE SHEETS', 'CONSOLIDATED STATEMENTS OF CASH'],
    'MDLZ': ['Consolidated Statements of Earn', 'Consolidated Balance Sheets, as', 'Consolidated Statements of Cash']
                   }

In [7]:
df_apple1 = pd.read_excel('AAPL_10-K_2014-10-27_Financial_Report.xlsx', sheet_name='CONSOLIDATED_STATEMENTS_OF_OPE')
df_apple2 = pd.read_excel('AAPL_10-K_2015-10-28_Financial_Report.xlsx', sheet_name='CONSOLIDATED_STATEMENTS_OF_OPE')
df_apple3 = pd.read_excel('AAPL_10-K_2016-10-26_Financial_Report.xlsx', sheet_name='CONSOLIDATED_STATEMENTS_OF_OPE')


ValueError: Worksheet named 'CONSOLIDATED_STATEMENTS_OF_OPE' not found

In [ ]:
df_apple1 = pd.read_excel('AAPL_10-K_2015-10-28_Financial_Report.xlsx', sheet_name=None)
df_apple2 = pd.read_excel('AAPL_10-K_2016-10-26_Financial_Report.xlsx', sheet_name=None)
df_apple3 = pd.read_excel('AAPL_10-K_2017-11-03_Financial_Report.xlsx', sheet_name=None)

value_list1 = list(df_apple1.values())
value_list2 = list(df_apple2.values())
value_list3 = list(df_apple3.values())

# [0] is usually the cover page which we don't need
value_list1[1]
value_list2[1]
value_list3[1]

,"CONSOLIDATED STATEMENTS OF OPERATIONS - USD ($) shares in Thousands, $ in Millions",12 Months Ended,Unnamed: 2,Unnamed: 3
0,NaN,"Sep. 30, 2017","Sep. 24, 2016","Sep. 26, 2015"
1,Income Statement [Abstract],NaN,NaN,NaN
2,Net sales,229234,215639,233715
3,Cost of sales,141048,131376,140089
4,Gross margin,88186,84263,93626
5,Operating expenses:,NaN,NaN,NaN
6,Research and development,11581,10045,8067
7,"Selling, general and administrative",15261,14194,14329
8,Total operating expenses,26842,24239,22396
9,Operating income,61344,60024,71230


In [37]:
merged = value_list1[1].merge(value_list2[1], on=value_list1[1].columns[0], how='left')
merged

,"CONSOLIDATED STATEMENTS OF OPERATIONS - USD ($) shares in Thousands, $ in Millions",12 Months Ended_x,Unnamed: 2_x,Unnamed: 3_x,12 Months Ended_y,Unnamed: 2_y,Unnamed: 3_y
0,NaN,"Sep. 26, 2015","Sep. 27, 2014","Sep. 28, 2013","Sep. 24, 2016","Sep. 26, 2015","Sep. 27, 2014"
1,Net sales,233715,182795,170910,215639,233715,182795
2,Cost of sales,140089,112258,106606,131376,140089,112258
3,Gross margin,93626,70537,64304,84263,93626,70537
4,Operating expenses:,NaN,NaN,NaN,NaN,NaN,NaN
5,Research and development,8067,6041,4475,10045,8067,6041
6,"Selling, general and administrative",14329,11993,10830,14194,14329,11993
7,Total operating expenses,22396,18034,15305,24239,22396,18034
8,Operating income,71230,52503,48999,60024,71230,52503
9,"Other income/(expense), net",1285,980,1156,1348,1285,980


In [40]:
merged.drop_duplicates(inplace=True, subset=merged.columns[0], keep='first')
merged

,"CONSOLIDATED STATEMENTS OF OPERATIONS - USD ($) shares in Thousands, $ in Millions",12 Months Ended_x,Unnamed: 2_x,Unnamed: 3_x,12 Months Ended_y,Unnamed: 2_y,Unnamed: 3_y
0,NaN,"Sep. 26, 2015","Sep. 27, 2014","Sep. 28, 2013","Sep. 24, 2016","Sep. 26, 2015","Sep. 27, 2014"
1,Net sales,233715,182795,170910,215639,233715,182795
2,Cost of sales,140089,112258,106606,131376,140089,112258
3,Gross margin,93626,70537,64304,84263,93626,70537
4,Operating expenses:,NaN,NaN,NaN,NaN,NaN,NaN
5,Research and development,8067,6041,4475,10045,8067,6041
6,"Selling, general and administrative",14329,11993,10830,14194,14329,11993
7,Total operating expenses,22396,18034,15305,24239,22396,18034
8,Operating income,71230,52503,48999,60024,71230,52503
9,"Other income/(expense), net",1285,980,1156,1348,1285,980


In [46]:
# /Users/jake/Development/Python/Financial4All/Financial4All/notebooks/excel_download_test.ipynb
// ... existing code ...
import pandas as pd
import glob

def process_income_statement(df):
    """
    Cleans and standardizes an income statement DataFrame.
    - Sets the first column (metrics) as the index.
    - Extracts years for column headers.
    - Cleans data by removing non-essential rows/columns and converting to numeric.
    - Handles duplicated metric names by ensuring every metric is unique.
    """
    # Use the first column for metrics, and drop rows if the metric is missing.
    metric_col_name = df.columns[0]
    df = df.rename(columns={metric_col_name: 'Metric'}).dropna(subset=['Metric'])

    # The first row typically contains the period-end dates for columns.
    date_row = df.iloc[0]
    new_columns = {'Metric': 'Metric'}
    for i, date_str in enumerate(date_row[1:]):
        if pd.notna(date_str):
            try:
                # Extract year from date string (e.g., 'Sep. 30, 2017')
                year = pd.to_datetime(date_str).year
                new_columns[df.columns[i+1]] = year
            except (ValueError, TypeError):
                # Ignore columns that don't have a valid date.
                pass

    df = df.rename(columns=new_columns)

    # Make sure all metric names are unique to prevent index errors.
    # We group by the metric name and append a counter to duplicates.
    df['Metric'] = df['Metric'].str.strip()
    df['cum_count'] = df.groupby('Metric').cumcount()
    df['Metric'] = df.apply(
        lambda row: f"{row['Metric']}_{row['cum_count']}" if row['cum_count'] > 0 else row['Metric'],
        axis=1
    )
    df = df.drop(columns='cum_count')

    # Keep only the columns we parsed a year for, plus the Metric column.
    year_cols = [c for c in df.columns if isinstance(c, int)]
    valid_columns = ['Metric'] + year_cols
    df = df[valid_columns]

    # Set the 'Metric' column as the index.
    df = df.set_index('Metric')

    # Convert all financial data to numeric types, coercing errors to NaN.
    df = df.apply(pd.to_numeric, errors='coerce')

    # Drop rows that are entirely empty (NaN).
    df = df.dropna(how='all')
    
    return df

# Find all Apple 10-K filings in the notebooks directory
# Assuming the notebook is run from the project root.
file_paths = glob.glob('notebooks/AAPL_10-K_*_Financial_Report.xlsx')
if not file_paths:
    # Fallback if run from notebooks/ directory.
    file_paths = glob.glob('AAPL_10-K_*_Financial_Report.xlsx')


# Process each file and store the cleaned DataFrame in a list
processed_statements = []
for path in sorted(file_paths):
    try:
        # Load the excel file, we are interested in the second sheet (index 1)
        excel_file = pd.read_excel(path, sheet_name=None, engine='openpyxl')
        # Check if the file has enough sheets
        if len(excel_file.values()) > 1:
            income_statement_df = list(excel_file.values())[1]
            
            # Process and clean the dataframe
            processed_df = process_income_statement(income_statement_df)
            
            # Final check for unique index before appending
            if not processed_df.index.is_unique:
                 print(f"Warning: Duplicate index found in {path} after processing. Skipping file.")
                 print(processed_df.index[processed_df.index.duplicated(keep=False)])
                 continue

            processed_statements.append(processed_df)
            print(f"Successfully processed: {path}")
        else:
            print(f"Could not find income statement sheet in {path}")
    except Exception as e:
        print(f"Could not process {path}: {e}")

# Combine all processed statements into one DataFrame
if processed_statements:
    # Concatenate all dataframes horizontally
    combined_df = pd.concat(processed_statements, axis=1)

    # The magic happens here: we group by column name (the year) and take the first
    # non-null value. This merges data for the same year from different files.
    # For example, the 2015 data from the 2015 and 2016 reports are consolidated.
    final_income_statement = combined_df.groupby(level=0, axis=1).first()

    # Sort the columns by year in descending order for readability
    final_income_statement = final_income_statement.reindex(sorted(final_income_statement.columns, reverse=True), axis=1)

    print("\nFinal Merged Income Statement:")
    display(final_income_statement)
else:
    print("No statements were processed.")

SyntaxError: invalid syntax (2907580899.py, line 2)